<a href="https://colab.research.google.com/github/lifeinapril/maya/blob/master/Maya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!pip install tensorflow_text  tensorflow_hub

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

In [54]:
df = pd.read_csv('./train.csv')
df.describe()

,review_count,rank,model_name
count,84437.000000,48730.000000,0.0
mean,3.027216,0.933409,NaN
std,0.324834,0.891965,NaN
min,0.000000,0.000000,NaN
25%,3.000000,0.000000,NaN
50%,3.000000,1.000000,NaN
75%,3.000000,2.000000,NaN
max,12.000000,15.000000,NaN


In [ ]:
prompters = df[df['role']=='prompter']
input = prompters["text"][:100].values

In [ ]:
assistants = df[df['role']=='assistant']
output = assistants['text'][1:101].values

In [57]:
train_input, val_input, train_output, val_output = train_test_split(input, output, test_size=0.2, random_state=42)


In [58]:
all_texts = list(train_input) + list(val_input) + list(train_output) + list(val_output)


In [59]:
tokenizer.fit_on_texts(all_texts)
# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

In [60]:
train_input_sequences = tokenizer.texts_to_sequences(train_input)
val_input_sequences = tokenizer.texts_to_sequences(val_input)
train_output_sequences = tokenizer.texts_to_sequences(train_output)
val_output_sequences = tokenizer.texts_to_sequences(val_output)

In [87]:
# Pad sequences
max_length = max(len(seq) for seq in train_input_sequences + val_input_sequences)
prompts_padded = pad_sequences(train_input_sequences, maxlen=max_length, padding='post')
responses_padded = pad_sequences(train_output_sequences, maxlen=max_length, padding='post')
val_input_padded = pad_sequences(val_input_sequences, maxlen=max_length, padding='post')
val_output_padded = pad_sequences(val_output_sequences, maxlen=max_length, padding='post')
prompts_padded
responses_padded

array([[   2,  291,  502, ...,   21,  765, 1173],
       [ 301,  347,  805, ...,    1,   50,  233],
       [2176, 2177,  412, ...,   20,   64, 1188],
       ...,
       [ 703,  215,  703, ...,  215,  201, 3474],
       [  35,  487, 3475, ...,    0,    0,    0],
       [1465,  455,  229, ...,  991, 3525,  229]], dtype=int32)

In [88]:
X_train = prompts_padded
y_train = responses_padded

In [109]:
def build(model_name, max_length, units=[128,64],vocab_size=300, activation='relu', output_activation='softmax'):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=vocab_size))
    # model.add(tf.keras.layers.LSTM(units=128))
    for units_count in units:
        model.add(tf.keras.layers.Dense(units_count, activation=activation))
    model.add(tf.keras.layers.Dense(vocab_size, activation=output_activation))
    model.save(f'drive/MyDrive/models/{model_name}.keras')
    return model

In [105]:
# Model Build
model = build("MAYA",max_length,vocab_size=vocab_size)

In [106]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 102, 3910)         15288100  
                                                                 
 dense_26 (Dense)            (None, 102, 128)          500608    
                                                                 
 dense_27 (Dense)            (None, 102, 64)           8256      
                                                                 
 dense_28 (Dense)            (None, 102, 3910)         254150    
                                                                 
Total params: 16051114 (61.23 MB)
Trainable params: 16051114 (61.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [107]:
# Train the model
model.fit(X_train, y_train, epochs=10)

Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    

In [ ]:
test_prompts = "hello world", "how are you", "test sentence"
test_responses = "good", "I'm fine", "this is a test"

test_prompts_sequences = tokenizer.texts_to_sequences(test_prompts)
test_responses_sequences = tokenizer.texts_to_sequences(test_responses)

test_prompts_padded = pad_sequences(test_prompts_sequences,maxlen=max_length, padding='post')
test_responses_padded = pad_sequences(test_responses_sequences, maxlen=max_length, padding='post')

# Evaluate the model
evaluation = model.evaluate(test_prompts_padded, test_responses_padded)
print("Test Loss:", evaluation[0])
print("Test MAE:", evaluation[1])

In [ ]:
new_prompt = ["Can you write a short introduction about the relevance of the term monopsony"]
# Tokenize and pad the new string
new_prompt_sequences = tokenizer.texts_to_sequences(new_prompt)
padded_sequences = pad_sequences(new_prompt_sequences, maxlen=model.input_shape[1], padding='post')
# Reshape using the Embedding layer
X_test = model.layers[0](padded_sequences)
# Make predictions
predictions = model.predict(X_test)
predictions

In [ ]:
predictions_strings = tokenizer.sequences_to_texts(predictions)
print(predictions_strings)